In [58]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import math
from scipy.signal import savgol_filter

from CCDPApy.helper import output_path

pd.set_option('display.max_columns', 200)

In [69]:
# loading data
data_path = output_path('genetech_raw_smoothed.csv')
seed_data_path = output_path('Genentech_seed_scale_smoothed.csv')

data_df = pd.read_csv(data_path)
seed_df = pd.read_csv(seed_data_path)

In [70]:
data_df.head(12)

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Feed Status,# Feed,base,osmo,vcd,dcd,tcd,IgG (mg/L),NH4+,Na+,Titer,IntvPCV,PCV,pCO2,pHoff,pO2,air sparge,air sparge sp,air sparge total,backpressure,co2 sparge,co2 sparge total,do2 controller output,do2 primary,do2 secondary,flowrate overlay,oxygen sparge,oxygen sparge total,ph controller output,ph primary,pressure exhaust valve,sparge total,temperature,temperature jacket,weight load cell,Titer_range,Titer_group,viab,Titer_category,Run Time (day),Run Time (hr),glc,lac,glc_after,lac_after,ccell,ivcc,IgG (mg),cglc,clac,mu,IgG (mg/10^9 cells/hr),qglc,qlac,mu_poly,qIgG_poly (mg/10^9 cells/hr),qglc_poly,qlac_poly,osmo_s,vcd_s,lac_s,mu_s
0,2023-01-01 00:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.000000,310.390602,0.707812,0.006987,0.714800,0,3.071507,46.345103,1.059949,0.000000,0.167421,93.000259,6.960743,101.258254,11.408122,5.688135,12734.85202,1.978115,3.245566,2785.682712,1.547035,29.921898,29.161844,250.984953,0.000000,0.000000,46.003623,7.028866,70.414004,11.278256,36.949926,37.437632,9452.84421,1.05-1.10,Middle 60%,99.022465,0,0,0.0,62.829937,5.794531,62.829937,5.794531,0.000000e+00,0.000000,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,0.127912,0.361730,301.028587,1.119630,9.053800,0.060916
1,2023-01-01 10:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.000000,294.969383,1.685265,0.051820,1.737085,0,3.621828,45.663156,1.059949,0.347610,0.258416,81.143342,7.002544,99.957496,53.190310,26.830021,26831.18445,1.981209,3.325126,1858.298578,7.096126,28.151353,28.057887,251.001728,0.000000,0.000000,45.862719,7.030608,70.302097,53.020003,37.042912,37.259285,10349.50443,1.05-1.10,Middle 60%,97.016840,0,0,10.0,62.126848,9.078070,62.126848,9.078070,1.172944e+07,11.965388,0,8437.068541,39402.468250,0.081690,0.0,0.058760,0.274420,0.064368,0,0.080913,0.138233,297.662207,1.496163,11.415765,0.046863
2,2023-01-01 20:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.000000,297.612555,1.544325,0.047093,1.591418,0,4.172149,44.981209,1.059949,1.803381,0.364611,69.286425,7.014704,98.656738,71.584015,35.751700,64305.34848,1.972489,3.253458,3794.494966,9.549251,30.234508,30.208355,250.950363,0.000000,0.000000,45.911384,7.028387,71.111027,71.403806,36.996902,37.371550,10343.89697,1.05-1.10,Middle 60%,97.040803,0,0,20.0,59.375606,13.064794,59.375606,13.064794,1.003815e+07,28.113340,0,41451.971625,87243.157993,-0.008728,0.0,0.170377,0.246887,0.043990,0,0.116140,0.136847,295.581111,1.662210,14.103841,0.035802
3,2023-01-02 06:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.000000,297.060884,1.858663,0.056206,1.914869,0,4.532507,44.299261,1.059949,3.521618,0.482487,59.669689,7.004803,94.844104,93.778188,46.761039,113614.71560,1.964130,3.507787,5868.770835,12.521825,28.366251,29.168578,250.965823,0.000000,0.000000,45.679751,7.031762,71.877170,93.974310,36.996902,37.388528,10338.88353,1.05-1.10,Middle 60%,97.064765,0,1,30.0,56.343132,16.252417,56.343132,16.252417,1.381021e+07,45.128280,0,77841.657230,125494.630417,0.018474,0.0,0.178224,0.187343,0.038360,0,0.118113,0.102851,295.169148,1.938018,16.836007,0.027269
4,2023-01-02 16:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.000000,293.574929,2.494772,0.104573,2.599345,0,4.699150,40.892881,1.059949,5.738774,0.634560,52.337375,6.976747,88.469988,125.706899,62.624315,178875.49560,1.965074,3.354045,7993.900298,16.706876,29.904632,30.813889,250.743895,0.000000,0.000000,45.847410,7.029856,72.944823,125.554414,37.041984,37.241820,10332.64942,1.05-1.10,Middle 60%,95.976960,0,1,40.0,53.598274,18.753849,53.598274,18.753849,2.144352e+07,66.895457,0,110779.951309,155511.813677,0.029223,0.0,0.126101,0.114918,0.027437,0,0.102968,0.069124,296.903417,2.439821,19.602439,0.020835
5,2023-01-03 02:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,32.615273,291.875212,2.736359

In [71]:
seed_df.head(12)

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),base,osmo,vcd,dcd,tcd,IgG (mg/L),NH4+,Na+,Titer,PCV,pCO2,pHoff,pO2,air sparge,air sparge sp,air sparge total,co2 sparge,co2 sparge total,do2 controller output,do2 primary,do2 secondary,flowrate overlay,oxygen sparge,oxygen sparge total,ph controller output,ph primary,pressure exhaust valve,sparge total,temperature,temperature jacket,weight load cell,Titer_range,Titer_group,viab,Titer_category,Run Time (day),Run Time (hr),glc,lac,glc_after,lac_after,ccell,ivcc,IgG (mg),cglc,clac,mu,IgG (mg/10^9 cells/hr),qglc,qlac,mu_poly,qIgG_poly (mg/10^9 cells/hr),qglc_poly,qlac_poly,mu_s
0,2022-01-01 00:00:00,80L,294592,Genentech,80000,0,65190.53925,65190.53925,0,0.000000,283.748784,0.415612,0.011999,0.427611,0,1.492941,91.042395,1.059949,0.066466,42.233436,7.175178,146.195805,0.009823,0.000000,125.059288,0.024126,25.129515,0.000000,91.364229,90.216909,5.002909,0.004518,0.000000,47.548051,7.228373,36.240140,0.013406,36.924890,37.418953,0.0,1.05-1.10,Middle 60%,97.194035,0,0,0,39.870958,3.868316,NaN,NaN,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,-1.143551,0.593985,-0.140583
1,2022-01-01 10:00:00,80L,294592,Genentech,80000,0,73579.87564,73579.87564,0,26.686667,279.139309,0.404801,0.012001,0.416802,0,1.939347,90.640228,1.059949,0.089396,35.275943,7.199856,130.109116,0.558427,11.044321,115.033526,0.025852,0.000000,11.046163,21.504282,41.253998,5.001564,0.003446,2.951405,47.439485,7.229691,36.224659,0.558755,37.057936,37.374289,0.0,1.05-1.10,Middle 60%,97.120687,0,0,10,38.752525,6.094060,38.752525,6.094060,2691.250847,4.102066,0,-252.196677,196.222586,0.009463,0.0,-0.886780,0.689963,-0.005622,0,-0.245786,0.683773,0.008572
2,2022-01-01 20:00:00,80L,294592,Genentech,80000,0,73244.03496,73244.03496,0,26.686667,279.286866,0.469224,0.018471,0.487695,0,2.385753,90.238061,1.059949,0.123071,28.318450,7.208973,114.022426,0.857580,17.275335,531.097068,0.022457,0.000000,17.289231,29.619363,42.900746,5.002751,0.007460,6.092490,47.784864,7.230197,36.524988,0.856754,36.962682,37.381836,0.0,1.05-1.10,Middle 60%,96.212574,0,0,20,37.851113,8.859019,37.851113,8.859019,7273.880481,8.472192,0,-173.158984,396.692710,0.014287,0.0,0.246403,0.624974,0.027589,0,0.269318,0.683590,0.027244
3,2022-01-02 06:00:00,80L,294592,Genentech,80000,0,72807.00722,72807.00722,0,26.686667,280.349446,0.650595,0.030442,0.681036,0,2.845994,90.520338,1.059949,0.166909,22.465542,7.192178,106.997621,0.784443,15.549562,1052.000825,0.004128,0.000000,18.359158,27.817933,36.044298,5.000931,0.061182,12.860067,49.382649,7.227377,35.976610,0.845073,37.006237,37.450892,0.0,1.05-1.10,Middle 60%,95.530097,0,1,30,36.069321,12.384662,36.069321,12.384662,20273.879437,14.071284,0,-26.890018,649.512579,0.031810,0.0,0.357907,0.618628,0.034112,0,0.395768,0.537813,0.018149
4,2022-01-02 16:00:00,80L,294592,Genentech,80000,0,72317.21588,72317.21588,0,27.263264,277.421355,0.881518,0.041143,0.922662,0,3.326306,91.795502,1.059949,0.222023,18.214995,7.164092,113.118408,0.926161,18.613675,1573.673546,0.000000,0.000000,21.101975,29.854183,34.690836,5.001971,0.067076,23.158474,50.000000,7.183341,35.788841,0.992384,37.006237,37.426133,0.0,1.05-1.10,Middle 60%,95.540811,0,1,40,33.272010,16.242056,33.272010,16.242056,36654.982867,21.731850,0,193.070147,922.402680,0.029484,0.0,0.395908,0.491177,0.024614,0,0.331794,0.412726,0.022042
5,2022-01-03 02:00:00,80L,294592,Genentech,80000,0,71745.96947,71745.96947,0,93.150458,279.057917,1.015951,0.062891,1.078842,0,3.806674,93.187186,1.059949,0.302025,14.249136,7.157986,118.518611,0.889960,17.765441,2097.747610,0.000000,0.000000,21.785619,30.118317,32.630495,5.002904,0.112281,44.036239,50.052650,7.176175,35.482364,0.995653,37.006237,37.320744,0.0,1.05-1.10,Middle 60%,94.170528,0,2,50,32.091407,19.803569,32.091407,19.803569,45796.437189,31.219198,0,296.780193,1168.648652,0.013380,0.0,0.151801,0.360432,0

In [72]:
# Matching IDs by titer values
id_list1 = data_df['ID'].unique()
id_list2 = seed_df['ID'].unique()
titer_list = data_df['Titer'].unique()

id_dict = {}
num_id = 0
for t in titer_list:
    id1 = data_df[data_df['Titer']==t]['ID'].unique()
    id2 = seed_df[seed_df['Titer']==t]['ID'].unique()

    if id1.size == 1:
        print(f'Titier: {t}')
        print(f'    ID1: {id1}    ID2: {id2}')
        id_dict[id2[0]] = id1[0]
        num_id += 1
    else:
        print(f'Titier: {t}')
        print(f'    ID1: {id1}    ID2: {id2}')
        for i, id2_i in enumerate(id2):
            id_dict[id2_i] = id1[i]

Titier: 1.059949311
    ID1: [295008 297879 300095]    ID2: [294592 297135 299317]
Titier: 1.151675694
    ID1: [295009 295579]    ID2: [294593 294594]
Titier: 0.988606569
    ID1: [295660]    ID2: [294595]
Titier: 1.110908413
    ID1: [295661 298178 302254 302720]    ID2: [294596 297442 301077 301946]
Titier: 1.029373851
    ID1: [295662 296335]    ID2: [294962 295668]
Titier: 1.090524772
    ID1: [295663 301815 301947 302335]    ID2: [294963 299940 301076 301814]
Titier: 1.049757491
    ID1: [295664 297162]    ID2: [294964 296581]
Titier: 1.00899021
    ID1: [295712 296159 296687]    ID2: [294965 295667 295670]
Titier: 0.876496546
    ID1: [295713]    ID2: [295666]
Titier: 1.080332952
    ID1: [296336]    ID2: [295669]
Titier: 0.845921085
    ID1: [296688]    ID2: [296073]
Titier: 0.907072007
    ID1: [296811 298177]    ID2: [295671 297136]
Titier: 1.039565671
    ID1: [296812 300769]    ID2: [296075 299569]
Titier: 0.927455648
    ID1: [297328 297878]    ID2: [296582 296754]
Titier:

In [73]:
# Replacing IDs
df_list = []
for id2, df_group in seed_df.groupby(by='ID'):
    new_id = id_dict[id2]
    df_group['ID'] = new_id
    df_list.append(df_group)

seed_df = pd.concat(df_list, axis=0)
seed_df = seed_df.reset_index(drop=True)
seed_df.head(12)

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),base,osmo,vcd,dcd,tcd,IgG (mg/L),NH4+,Na+,Titer,PCV,pCO2,pHoff,pO2,air sparge,air sparge sp,air sparge total,co2 sparge,co2 sparge total,do2 controller output,do2 primary,do2 secondary,flowrate overlay,oxygen sparge,oxygen sparge total,ph controller output,ph primary,pressure exhaust valve,sparge total,temperature,temperature jacket,weight load cell,Titer_range,Titer_group,viab,Titer_category,Run Time (day),Run Time (hr),glc,lac,glc_after,lac_after,ccell,ivcc,IgG (mg),cglc,clac,mu,IgG (mg/10^9 cells/hr),qglc,qlac,mu_poly,qIgG_poly (mg/10^9 cells/hr),qglc_poly,qlac_poly,mu_s
0,2022-01-01 00:00:00,80L,295008,Genentech,80000,0,65190.53925,65190.53925,0,0.000000,283.748784,0.415612,0.011999,0.427611,0,1.492941,91.042395,1.059949,0.066466,42.233436,7.175178,146.195805,0.009823,0.000000,125.059288,0.024126,25.129515,0.000000,91.364229,90.216909,5.002909,0.004518,0.000000,47.548051,7.228373,36.240140,0.013406,36.924890,37.418953,0.000000,1.05-1.10,Middle 60%,97.194035,0,0,0,39.870958,3.868316,NaN,NaN,0.000000,0.000000,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,-1.143551,0.593985,-0.140583
1,2022-01-01 10:00:00,80L,295008,Genentech,80000,0,73579.87564,73579.87564,0,26.686667,279.139309,0.404801,0.012001,0.416802,0,1.939347,90.640228,1.059949,0.089396,35.275943,7.199856,130.109116,0.558427,11.044321,115.033526,0.025852,0.000000,11.046163,21.504282,41.253998,5.001564,0.003446,2.951405,47.439485,7.229691,36.224659,0.558755,37.057936,37.374289,0.000000,1.05-1.10,Middle 60%,97.120687,0,0,10,38.752525,6.094060,38.752525,6.094060,2691.250847,4.102066,0,-252.196677,196.222586,0.009463,0.0,-0.886780,0.689963,-0.005622,0,-0.245786,0.683773,0.008572
2,2022-01-01 20:00:00,80L,295008,Genentech,80000,0,73244.03496,73244.03496,0,26.686667,279.286866,0.469224,0.018471,0.487695,0,2.385753,90.238061,1.059949,0.123071,28.318450,7.208973,114.022426,0.857580,17.275335,531.097068,0.022457,0.000000,17.289231,29.619363,42.900746,5.002751,0.007460,6.092490,47.784864,7.230197,36.524988,0.856754,36.962682,37.381836,0.000000,1.05-1.10,Middle 60%,96.212574,0,0,20,37.851113,8.859019,37.851113,8.859019,7273.880481,8.472192,0,-173.158984,396.692710,0.014287,0.0,0.246403,0.624974,0.027589,0,0.269318,0.683590,0.027244
3,2022-01-02 06:00:00,80L,295008,Genentech,80000,0,72807.00722,72807.00722,0,26.686667,280.349446,0.650595,0.030442,0.681036,0,2.845994,90.520338,1.059949,0.166909,22.465542,7.192178,106.997621,0.784443,15.549562,1052.000825,0.004128,0.000000,18.359158,27.817933,36.044298,5.000931,0.061182,12.860067,49.382649,7.227377,35.976610,0.845073,37.006237,37.450892,0.000000,1.05-1.10,Middle 60%,95.530097,0,1,30,36.069321,12.384662,36.069321,12.384662,20273.879437,14.071284,0,-26.890018,649.512579,0.031810,0.0,0.357907,0.618628,0.034112,0,0.395768,0.537813,0.018149
4,2022-01-02 16:00:00,80L,295008,Genentech,80000,0,72317.21588,72317.21588,0,27.263264,277.421355,0.881518,0.041143,0.922662,0,3.326306,91.795502,1.059949,0.222023,18.214995,7.164092,113.118408,0.926161,18.613675,1573.673546,0.000000,0.000000,21.101975,29.854183,34.690836,5.001971,0.067076,23.158474,50.000000,7.183341,35.788841,0.992384,37.006237,37.426133,0.000000,1.05-1.10,Middle 60%,95.540811,0,1,40,33.272010,16.242056,33.272010,16.242056,36654.982867,21.731850,0,193.070147,922.402680,0.029484,0.0,0.395908,0.491177,0.024614,0,0.331794,0.412726,0.022042
5,2022-01-03 02:00:00,80L,295008,Genentech,80000,0,71745.96947,71745.96947,0,93.150458,279.057917,1.015951,0.062891,1.078842,0,3.806674,93.187186,1.059949,0.302025,14.249136,7.157986,118.518611,0.889960,17.765441,2097.747610,0.000000,0.000000,21.785619,30.118317,32.630495,5.002904,0.112281,44.036239,50.052650,7.176175,35.482364,0.995653,37.006237,37.320744,0.000000,1.05-1.10,Middle 60%,94.170528,0,2,50,32.091407,19.803569,32.091407,19.803569,45796.437189,31.219198,0,296.780193,1168.648652,0.

In [74]:
# Check titer values by IDs
for id in data_df['ID'].unique():
    t1 = data_df[data_df['ID']==id]['Titer'].iat[0]
    t2 = seed_df[seed_df['ID']==id]['Titer'].iat[0]
    if t1!=t2:
        print(f'ID: {id} is wrong')

In [75]:
# Glucose Feed
data_df['glc_feed'] = data_df['glc_after'] - data_df['glc']
seed_df['glc_feed'] = 0.0

# time step 
data_df['input_step'] = data_df['Run Time (hr)'].apply(lambda x: int(x/10))
seed_df['input_step'] = seed_df['Run Time (hr)'].apply(lambda x: int(x/10))

data_df['input_step_2000'] = data_df['Run Time (hr)'].apply(lambda x: int(x/10))
seed_df['input_step_2000'] = seed_df['Run Time (hr)'].apply(lambda x: int(x/10))

data_df['input_step_400_2000'] = data_df['Run Time (hr)'].apply(lambda x: int(x/10))
seed_df['input_step_400_2000'] = seed_df['Run Time (hr)'].apply(lambda x: int(x/10))

data_df['input_step_80_400_2000'] = data_df['Run Time (hr)'].apply(lambda x: int(x/10))
seed_df['input_step_80_400_2000'] = seed_df['Run Time (hr)'].apply(lambda x: int(x/10))

In [76]:
data_df.head()

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Feed Status,# Feed,base,osmo,vcd,dcd,tcd,IgG (mg/L),NH4+,Na+,Titer,IntvPCV,PCV,pCO2,pHoff,pO2,air sparge,air sparge sp,air sparge total,backpressure,co2 sparge,co2 sparge total,do2 controller output,do2 primary,do2 secondary,flowrate overlay,oxygen sparge,oxygen sparge total,ph controller output,ph primary,pressure exhaust valve,sparge total,temperature,temperature jacket,weight load cell,Titer_range,Titer_group,viab,Titer_category,Run Time (day),Run Time (hr),glc,lac,glc_after,lac_after,ccell,ivcc,IgG (mg),cglc,clac,mu,IgG (mg/10^9 cells/hr),qglc,qlac,mu_poly,qIgG_poly (mg/10^9 cells/hr),qglc_poly,qlac_poly,osmo_s,vcd_s,lac_s,mu_s,glc_feed,input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000
0,2023-01-01 00:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.0,310.390602,0.707812,0.006987,0.714800,0,3.071507,46.345103,1.059949,0.000000,0.167421,93.000259,6.960743,101.258254,11.408122,5.688135,12734.85202,1.978115,3.245566,2785.682712,1.547035,29.921898,29.161844,250.984953,0.0,0.0,46.003623,7.028866,70.414004,11.278256,36.949926,37.437632,9452.84421,1.05-1.10,Middle 60%,99.022465,0,0,0.0,62.829937,5.794531,62.829937,5.794531,0.000000e+00,0.000000,0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,0.127912,0.361730,301.028587,1.119630,9.053800,0.060916,0.0,0,0,0,0
1,2023-01-01 10:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.0,294.969383,1.685265,0.051820,1.737085,0,3.621828,45.663156,1.059949,0.347610,0.258416,81.143342,7.002544,99.957496,53.190310,26.830021,26831.18445,1.981209,3.325126,1858.298578,7.096126,28.151353,28.057887,251.001728,0.0,0.0,45.862719,7.030608,70.302097,53.020003,37.042912,37.259285,10349.50443,1.05-1.10,Middle 60%,97.016840,0,0,10.0,62.126848,9.078070,62.126848,9.078070,1.172944e+07,11.965388,0,8437.068541,39402.468250,0.081690,0.0,0.058760,0.274420,0.064368,0,0.080913,0.138233,297.662207,1.496163,11.415765,0.046863,0.0,1,1,1,1
2,2023-01-01 20:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.0,297.612555,1.544325,0.047093,1.591418,0,4.172149,44.981209,1.059949,1.803381,0.364611,69.286425,7.014704,98.656738,71.584015,35.751700,64305.34848,1.972489,3.253458,3794.494966,9.549251,30.234508,30.208355,250.950363,0.0,0.0,45.911384,7.028387,71.111027,71.403806,36.996902,37.371550,10343.89697,1.05-1.10,Middle 60%,97.040803,0,0,20.0,59.375606,13.064794,59.375606,13.064794,1.003815e+07,28.113340,0,41451.971625,87243.157993,-0.008728,0.0,0.170377,0.246887,0.043990,0,0.116140,0.136847,295.581111,1.662210,14.103841,0.035802,0.0,2,2,2,2
3,2023-01-02 06:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.0,297.060884,1.858663,0.056206,1.914869,0,4.532507,44.299261,1.059949,3.521618,0.482487,59.669689,7.004803,94.844104,93.778188,46.761039,113614.71560,1.964130,3.507787,5868.770835,12.521825,28.366251,29.168578,250.965823,0.0,0.0,45.679751,7.031762,71.877170,93.974310,36.996902,37.388528,10338.88353,1.05-1.10,Middle 60%,97.064765,0,1,30.0,56.343132,16.252417,56.343132,16.252417,1.381021e+07,45.128280,0,77841.657230,125494.630417,0.018474,0.0,0.178224,0.187343,0.038360,0,0.118113,0.102851,295.169148,1.938018,16.836007,0.027269,0.0,3,3,3,3
4,2023-01-02 16:00:00,CL G,295008,Genetech,12000000,0,12000000,12000000,0,0,2,0.0,293.574929,2.494772,0.104573,2.599345,0,4.699150,40.892881,1.059949,5.738774,0.634560,52.337375,6.976747,88.469988,125.706899,62.624315,178875.49560,1.965074,3.354045,7993.900298,16.706876,29.904632,30.813889,250.743895,0.0,0.0,45.847410,7.029856,72.944823,125.554414,37.041984,37.241820,10332.64942,1.05-1.10,Middle 60%,95.976960,0,1,40.0,53.598274,18.753849,53.598274,18.753849,2.144352e+07,66.895457,0,110779.951309,155511.813677,0.029223,0.0,0.126101,0.114918,0.027437,0,0.102968,0.069124,296.903417,2.439821,19.602439,0.020835,0.0,4,4,4,4


In [79]:
params_to_use_in_prod = [
    'Cell Line', 'ID', 'Titer_group', 'Titer_category', 
    'Run Time (hr)', 
    'input_step', 'input_step_2000', 'input_step_400_2000', 'input_step_80_400_2000',
    'glc', 'glc_after', 'glc_feed',
    'osmo_s', 'vcd_s', 'lac_s', 
    'mu_s',  'qglc_poly', 'qlac_poly', 'base'
]

data_df = data_df[params_to_use_in_prod]

# Rename
data_df = data_df.rename(columns={'osmo_s': 'osmo',
                                  'vcd_s': 'vcd',
                                  'lac_s': 'lac',
                                  'mu_s': 'mu'})

data_df.head()

,Cell Line,ID,Titer_group,Titer_category,Run Time (hr),input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000,glc,glc_after,glc_feed,osmo,vcd,lac,mu,qglc_poly,qlac_poly,base
0,CL G,295008,Middle 60%,0,0.0,0,0,0,0,62.829937,62.829937,0.0,301.028587,1.119630,9.053800,0.060916,0.127912,0.361730,0.0
1,CL G,295008,Middle 60%,0,10.0,1,1,1,1,62.126848,62.126848,0.0,297.662207,1.496163,11.415765,0.046863,0.080913,0.138233,0.0
2,CL G,295008,Middle 60%,0,20.0,2,2,2,2,59.375606,59.375606,0.0,295.581111,1.662210,14.103841,0.035802,0.116140,0.136847,0.0
3,CL G,295008,Middle 60%,0,30.0,3,3,3,3,56.343132,56.343132,0.0,295.169148,1.938018,16.836007,0.027269,0.118113,0.102851,0.0
4,CL G,295008,Middle 60%,0,40.0,4,4,4,4,53.598274,53.598274,0.0,296.903417,2.439821,19.602439,0.020835,0.102968,0.069124,0.0


In [81]:
parameters_to_use_in_seed = [
    'Cell Line', 'ID', 'Titer_group', 'Titer_category',
    'Run Time (hr)', 
    'input_step', 'input_step_2000', 'input_step_400_2000', 'input_step_80_400_2000',
    'glc', 'glc_after', 'glc_feed',
    'osmo', 'vcd', 'lac', 
    'mu_s',  'qglc_poly', 'qlac_poly', 'base'
]

seed_df = seed_df[parameters_to_use_in_seed]
seed_df = seed_df.rename(columns={'mu_s': 'mu'})

seed_df.head(8)

,Cell Line,ID,Titer_group,Titer_category,Run Time (hr),input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000,glc,glc_after,glc_feed,osmo,vcd,lac,mu,qglc_poly,qlac_poly,base
0,80L,295008,Middle 60%,0,0,0,0,0,0,39.870958,NaN,0.0,283.748784,0.415612,3.868316,-0.140583,-1.143551,0.593985,0.000000
1,80L,295008,Middle 60%,0,10,1,1,1,1,38.752525,38.752525,0.0,279.139309,0.404801,6.094060,0.008572,-0.245786,0.683773,26.686667
2,80L,295008,Middle 60%,0,20,2,2,2,2,37.851113,37.851113,0.0,279.286866,0.469224,8.859019,0.027244,0.269318,0.683590,26.686667
3,80L,295008,Middle 60%,0,30,3,3,3,3,36.069321,36.069321,0.0,280.349446,0.650595,12.384662,0.018149,0.395768,0.537813,26.686667
4,80L,295008,Middle 60%,0,40,4,4,4,4,33.272010,33.272010,0.0,277.421355,0.881518,16.242056,0.022042,0.331794,0.412726,27.263264
5,80L,295008,Middle 60%,0,50,5,5,5,5,32.091407,32.091407,0.0,279.057917,1.015951,19.803569,0.017720,0.225963,0.356712,93.150458
6,80L,295008,Middle 60%,0,60,6,6,6,6,29.574468,29.574468,0.0,281.625678,0.945606,23.578892,-0.077981,0.069748,0.363970,190.915169
7,400L,295008,Middle 60%,0,0,0,0,0,0,57.363282,NaN,0.0,278.459512,0.366848,4.398492,-0.093513,0.195301,0.227062,109.353334


In [85]:
seed_80L = seed_df[seed_df['Cell Line']=='80L'].copy()
seed_400L = seed_df[seed_df['Cell Line']=='400L'].copy()
seed_2000L = seed_df[seed_df['Cell Line']=='2000L'].copy()
prod_data = data_df.copy()

In [86]:
# get time steps for each seed data
seed_input_80 = seed_80L['input_step'].max() + 1
seed_input_400 = seed_400L['input_step'].max() + 1
seed_input_2000 = seed_2000L['input_step'].max() + 1

seed_input_80, seed_input_400, seed_input_2000

(7, 7, 7)

In [87]:
# shift time for all seed train data
seed_400L['input_step_80_400_2000'] = seed_400L['input_step_80_400_2000'].apply(lambda x: x + seed_input_80)
seed_2000L['input_step_80_400_2000'] = seed_2000L['input_step_80_400_2000'].apply(lambda x: x + (seed_input_80 + seed_input_400))
prod_data['input_step_80_400_2000'] = prod_data['input_step_80_400_2000'].apply(lambda x: x + (seed_input_80 + seed_input_400 + seed_input_2000))

prod_data.head()

,Cell Line,ID,Titer_group,Titer_category,Run Time (hr),input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000,glc,glc_after,glc_feed,osmo,vcd,lac,mu,qglc_poly,qlac_poly,base
0,CL G,295008,Middle 60%,0,0.0,0,0,0,21,62.829937,62.829937,0.0,301.028587,1.119630,9.053800,0.060916,0.127912,0.361730,0.0
1,CL G,295008,Middle 60%,0,10.0,1,1,1,22,62.126848,62.126848,0.0,297.662207,1.496163,11.415765,0.046863,0.080913,0.138233,0.0
2,CL G,295008,Middle 60%,0,20.0,2,2,2,23,59.375606,59.375606,0.0,295.581111,1.662210,14.103841,0.035802,0.116140,0.136847,0.0
3,CL G,295008,Middle 60%,0,30.0,3,3,3,24,56.343132,56.343132,0.0,295.169148,1.938018,16.836007,0.027269,0.118113,0.102851,0.0
4,CL G,295008,Middle 60%,0,40.0,4,4,4,25,53.598274,53.598274,0.0,296.903417,2.439821,19.602439,0.020835,0.102968,0.069124,0.0


In [88]:
# shift time by time in 2000L data
prod_data['input_step_2000'] = prod_data['input_step_2000'].apply(lambda x: x + seed_input_2000)

In [89]:
# shif time by time in 400L and 2000L
seed_2000L['input_step_400_2000'] = seed_2000L['input_step_400_2000'].apply(lambda x: x + seed_input_400)
prod_data['input_step_400_2000'] = prod_data['input_step_400_2000'].apply(lambda x: x + (seed_input_400 + seed_input_2000))

In [90]:
# Combine seed and production data
data_all = pd.concat([seed_80L, seed_400L, seed_2000L, prod_data], axis=0).sort_values(by=['ID', 'input_step_80_400_2000'], ascending=[True, True])
data_all = data_all.reset_index(drop=True)
data_all.head(50)

,Cell Line,ID,Titer_group,Titer_category,Run Time (hr),input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000,glc,glc_after,glc_feed,osmo,vcd,lac,mu,qglc_poly,qlac_poly,base
0,80L,295008,Middle 60%,0,0.0,0,0,0,0,39.870958,NaN,0.000000,283.748784,0.415612,3.868316,-0.140583,-1.143551,0.593985,0.000000
1,80L,295008,Middle 60%,0,10.0,1,1,1,1,38.752525,38.752525,0.000000,279.139309,0.404801,6.094060,0.008572,-0.245786,0.683773,26.686667
2,80L,295008,Middle 60%,0,20.0,2,2,2,2,37.851113,37.851113,0.000000,279.286866,0.469224,8.859019,0.027244,0.269318,0.683590,26.686667
3,80L,295008,Middle 60%,0,30.0,3,3,3,3,36.069321,36.069321,0.000000,280.349446,0.650595,12.384662,0.018149,0.395768,0.537813,26.686667
4,80L,295008,Middle 60%,0,40.0,4,4,4,4,33.272010,33.272010,0.000000,277.421355,0.881518,16.242056,0.022042,0.331794,0.412726,27.263264
5,80L,295008,Middle 60%,0,50.0,5,5,5,5,32.091407,32.091407,0.000000,279.057917,1.015951,19.803569,0.017720,0.225963,0.356712,93.150458
6,80L,295008,Middle 60%,0,60.0,6,6,6,6,29.574468,29.574468,0.000000,281.625678,0.945606,23.578892,-0.077981,0.069748,0.363970,190.915169
7,400L,295008,Middle 60%,0,0.0,0,0,0,7,57.363282,NaN,0.000000,278.459512,0.366848,4.398492,-0.093513,0.195301,0.227062,109.353334
8,400L,295008,Middle 60%,0,10.0,1,1,1,8,57.076529,57.076529,0.000000,284.313171,0.499491,5.970594,0.030049,0.090327,0.336018,0.000000
9,400L,295008,Middle 60%,0,20.0,2,2,2,9,55.929696,55.929696,0.000000,286.000488,0.580184,7.769178,0.017810,0.082511,0.398433,0.000000


In [91]:
data_all['Run Time (hr)'] = data_all['Run Time (hr)'].astype(int)
data_all.head()

,Cell Line,ID,Titer_group,Titer_category,Run Time (hr),input_step,input_step_2000,input_step_400_2000,input_step_80_400_2000,glc,glc_after,glc_feed,osmo,vcd,lac,mu,qglc_poly,qlac_poly,base
0,80L,295008,Middle 60%,0,0,0,0,0,0,39.870958,NaN,0.0,283.748784,0.415612,3.868316,-0.140583,-1.143551,0.593985,0.000000
1,80L,295008,Middle 60%,0,10,1,1,1,1,38.752525,38.752525,0.0,279.139309,0.404801,6.094060,0.008572,-0.245786,0.683773,26.686667
2,80L,295008,Middle 60%,0,20,2,2,2,2,37.851113,37.851113,0.0,279.286866,0.469224,8.859019,0.027244,0.269318,0.683590,26.686667
3,80L,295008,Middle 60%,0,30,3,3,3,3,36.069321,36.069321,0.0,280.349446,0.650595,12.384662,0.018149,0.395768,0.537813,26.686667
4,80L,295008,Middle 60%,0,40,4,4,4,4,33.272010,33.272010,0.0,277.421355,0.881518,16.242056,0.022042,0.331794,0.412726,27.263264


In [93]:
data_all.dtypes

Cell Line                  object
ID                          int64
Titer_group                object
Titer_category              int64
Run Time (hr)               int64
input_step                  int64
input_step_2000             int64
input_step_400_2000         int64
input_step_80_400_2000      int64
glc                       float64
glc_after                 float64
glc_feed                  float64
osmo                      float64
vcd                       float64
lac                       float64
mu                        float64
qglc_poly                 float64
qlac_poly                 float64
base                      float64
dtype: object

In [94]:
# Saving
path = output_path('Genentech_seed_train.csv')
data_all.to_csv(path, index=False)